[![PyBind11](pybind11.png)](http://pybind11.readthedocs.io/en/stable/)

* Similar to Boost::Python, but easier to build
* Pure C++11 (no new language required), no dependencies
* Builds remain simple and don't require preprocessing
* Easy to customize result
* Great Gitter community
* Used in [GooFit 2.1+](https://goofit.github.io) for CUDA too [[CHEP talk]](https://indico.cern.ch/event/587955/contributions/2938087/)

Downsides:
* Still verbose
* Development variable

In [ ]:
%%writefile pybindclass.cpp

#include <pybind11/pybind11.h>
#include "SimpleClass.hpp"

namespace py = pybind11;

PYBIND11_MODULE(pybindclass, m) {
    py::class_<Simple>(m, "Simple")
        .def(py::init<int>())
        .def("get", &Simple::get)
    ;
}

In [ ]:
!c++ -std=c++11 pybindclass.cpp -shared {inc} -o pybindclass.so {plat}

In [ ]:
import pybindclass
x = pybindclass.Simple(4)
x.get()